In [37]:

import cv2
import pytesseract

# === LOAD IMAGE ===
image_path = "/Users/tanmaysingh/Desktop/Unknown.jpeg"
img = cv2.imread(image_path)

# Check if the image was successfully loaded
if img is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

# === PREPROCESSING ===
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Define kernel for dilation
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))
dilation = cv2.dilate(thresh, rect_kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# Copy original image to draw rectangles
im2 = img.copy()

# === TEXT DETECTION AND EXTRACTION ===
output_file_path = "/Users/tanmaysingh/Desktop/aipr/recognized.txt"
with open(output_file_path, "w") as file:
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cropped = gray[y:y + h, x:x + w]  # Use grayscale for better OCR
        
        text = pytesseract.image_to_string(cropped)
        cleaned_text = text.strip()
        
        if cleaned_text:
            file.write(cleaned_text + "\n")
            # Draw rectangle around detected text
            cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)

print(f"Text extraction complete. Output saved to: {output_file_path}")





Text extraction complete. Output saved to: /Users/tanmaysingh/Desktop/aipr/recognized.txt


In [43]:

import pyttsx3
import os
import sys

def text_to_speech(file_path):
    # Check if the file exists
    if not os.path.isfile(file_path):
        print(f"Error: '{file_path}' not found.")
        return

    # Initialize the pyttsx3 engine
    try:
        engine = pyttsx3.init()
    except Exception as e:
        print(f"Error initializing the speech engine: {e}")
        return

    # Set properties for the speech engine
    engine.setProperty('rate', 150)
    engine.setProperty('volume', 1.0)

    # Read the text from the file
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            text_to_speak = file.read().strip()
    except Exception as e:
        print(f"Error reading the file: {e}")
        return

    if text_to_speak:
        print("Reading the text aloud...")
        engine.say(text_to_speak)
        engine.runAndWait()
    else:
        print("The file is empty. No text to speak.")

if __name__ == "__main__":
    # Filter out unintended args like '-f'
    valid_args = [arg for arg in sys.argv[1:] if not arg.startswith("-")]

    if valid_args:
        file_path = valid_args[0]
    else:
        file_path = "/Users/tanmaysingh/Desktop/aipr/recognized.txt"

    text_to_speech(file_path)



Reading the text aloud...
